In [7]:
using Pkg
Pkg.activate(@__DIR__)

"/home/twan/.julia/dev/CentroidalTrajOpt/notebooks/Project.toml"

To do:

* Environment data should have its own axis, not shared with contacts
* Integer variables for contact pairs
* Initial condition modification
* Test that CoPs are in region
* Test that friction cone constraints are satisfied
* Move utilities to appropriate places
* Make foot locations variables
* Fix CoPs during each piece? Try not to.
* Constrain each CoP to be within a circle around the end effector position (to avoid rotation-dependence issues)

In [8]:
using CentroidalTrajOpt
using LinearAlgebra
using SCIP

In [9]:
# Initial conditions
c0 = SVector(0.05, 0.1, 1.0)
ċ0 = SVector(0.25, 0.15, 0.1)
g = SVector(0.0, 0.0, -9.81);

In [10]:
problem = CentroidalTrajectoryProblem(with_optimizer(SCIP.Optimizer, limits_gap=0.05), c0=c0, ċ0=ċ0, g=g);

In [11]:
result = solve!(problem);

presolving:
(round 1, fast)       116 del vars, 157 del conss, 0 add conss, 126 chg bounds, 0 chg sides, 0 chg coeffs, 8 upgd conss, 0 impls, 0 clqs
(round 2, fast)       116 del vars, 157 del conss, 0 add conss, 136 chg bounds, 0 chg sides, 0 chg coeffs, 8 upgd conss, 0 impls, 0 clqs
(round 3, fast)       116 del vars, 157 del conss, 0 add conss, 160 chg bounds, 0 chg sides, 0 chg coeffs, 8 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 116 deleted vars, 157 deleted constraints, 0 added constraints, 160 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 64 variables (0 bin, 0 int, 0 impl, 64 cont) and 54 constraints
      8 constraints of type <soc>
      4 constraints of type <SOS1>
      9 constraints of type <linear>
     33 constraints of type <quadratic>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n| 

In [12]:
using Test
c = result.center_of_mass
ċ = map_subfunctions(x -> map_elements(derivative, x), c)
c̈ = map_subfunctions(x -> map_elements(derivative, x), ċ)
rs = result.centers_of_pressure
fs = result.contact_forces
τns = result.contact_normal_torques
ns = normals(problem)

# Times
@test issorted(result.break_times)
@test all(x -> x >= 0, result.break_times)

T = last(result.break_times)

# Initial and final conditions
@test c(0) ≈ c0 atol=1e-12
@test ċ(0) ≈ ċ0 atol=1e-12
@test ċ(T) ≈ zeros(3) atol=1e-12
@test c̈(T) ≈ zeros(3) atol=1e-12

for t in range(0, T, length=100)
    ftot = sum(fs[contact](t) for contact in problem.contacts.val)
    
    # Dynamics
    @test c̈(t) ≈ g + ftot atol=1e-5
    
    # Torque about CoM
    n = ns[problem.regions(1)]
    τ = sum((rs[j](t) - c(t)) × fs[j](t) + n * τns[j](t) for j in problem.contacts.val)
    @test τ ≈ zeros(3) atol=1e-4
    
    # Friction cones
    for j in problem.contacts.val
        f = fs[j]
        τn = τns[j]
        
        f_t = f(t)
        fn_t = f_t ⋅ n
        τn_t = τn(t)
        @test fn_t >= 0
        
        # TODO: μ checks
#         @test norm(f_t - n * fn_t) <= μ * fn_t + 1e-6
#         @test -μrot * fn_t <= τn_t
#         @test τn_t <= μrot * fn_s
    end
end

# Continuity around breaks
for t in result.break_times
    @test c(t - 1e-8) ≈ c(t + 1e-8) atol=1e-5
    @test ċ(t - 1e-8) ≈ ċ(t + 1e-8) atol=1e-5
end